`TokenCrossEntropyLoss`和`LabelSmoothingLoss`都是用于计算神经网络模型的损失函数（loss function）的。在机器翻译任务中，通常使用交叉熵损失函数来衡量翻译结果与目标语言标准答案之间的差距。而`TokenCrossEntropyLoss`是交叉熵的一种实现方式，对于每个序列的每个位置，它计算目标序列与模型预测序列之间的交叉熵。

`LabelSmoothingLoss`则是一种对交叉熵损失函数的改进，在训练过程中使用平滑的标签（smoothed labels），以使得模型更加稳健，可以减轻模型对噪声和不准确标签的敏感性。具体地，它将标签中的原始值进行平滑（如将目标单词的one-hot向量中的1分散到其他单词的向量中），以减少对标签过于自信的预测。

使用哪种损失函数取决于具体的任务和需求，但在机器翻译领域中，目前较为常用的损失函数就是交叉熵损失函数和平滑标签交叉熵损失函数。

In [1]:
#     trainer = EpochSeq2SeqTrainer(
#         model=model,
#         train_dataloader=train_dataloader,
#         val_dataloader=val_dataloader,
#         loss_function=loss_function,
#         metric_function=accuracy_function,
#         optimizer=optimizer,
#         logger=logger,
#         run_name=run_name,
#         save_config=config['save_config'],
#         save_checkpoint=config['save_checkpoint'],
#         config=config
#     )

# 损失函数loss_function

# from losses import TokenCrossEntropyLoss, LabelSmoothingLoss
#     if config['label_smoothing'] > 0.0:
#         loss_function = LabelSmoothingLoss(label_smoothing=config['label_smoothing'],
#                                            vocabulary_size=target_dictionary.vocabulary_size)
#     else:
#         loss_function = TokenCrossEntropyLoss()

# 所以导出损失函数的计算，就是这下面的两类
import torch
from torch import nn


class TokenCrossEntropyLoss(nn.Module):#计算token的交叉熵损失

    def __init__(self, pad_index=0):
        super(TokenCrossEntropyLoss, self).__init__()

        self.pad_index = pad_index # 哦，我理解错了，这个是填充词的符号，用来建立类似于掩码层的，
        #用来把值为pad_index的，也就是值为0的数据给做成掩码层的
        self.base_loss_function = nn.CrossEntropyLoss(reduction='sum', ignore_index=pad_index)#计算交叉熵损失
        """
        1. `reduction`: 用于指定如何计算最终的损失值，该参数共有三个可选值：

           - `sum`：对每个样本的损失值求和，最后再对所有样本的损失值求和。
           - `mean`：对每个样本的损失值求平均值，最后对所有样本的平均损失值求和。
           - `none`：不对损失值进行聚合，直接返回每个样本的损失值。

           在这里，`reduction='sum'` 表示对所有样本的损失值求和。

        2. `ignore_index`: 用于指定哪个索引值的样本应该忽略计算损失值，即将其损失值设为 0。
        在这里，`ignore_index=pad_index` 表示将 PAD token 对应的索引值所在的位置的损失值设为 0，以避免将 PAD token 考虑进计算中。
        """

    def forward(self, outputs, targets):
        batch_size, seq_len, vocabulary_size = outputs.size()

        outputs_flat = outputs.view(batch_size * seq_len, vocabulary_size)# 展平为二维(64*49，35820)
        targets_flat = targets.view(batch_size * seq_len)#目标展平为一维，64*49

        batch_loss = self.base_loss_function(outputs_flat, targets_flat)
        #`nn.CrossEntropyLoss`要求输入的样本形状为 `(N, C)`，其中 `N` 是样本数量，`C` 是分数的类别数目。
        #将标签张量$T$的维度转换为N。此时，T_i表示第i个样本对应的真实类别的索引。
        #所以这里的输入是每个位置的每个类别数据的具体概率分布（未归一化），目标是onehot编码就行了，代表这块位置本来的类别号
        #输出向量不需要进行归一化，而可以直接用来计算损失值。
        count = (targets != self.pad_index).sum().item()
        #在这个损失函数中，count表示除去padding token后真正参与计算损失的单词数目。

        return batch_loss, count

#这块还是熟悉的，嫌麻烦就不步进了

class LabelSmoothingLoss(nn.Module):
    """
    With label smoothing,
    KL-divergence between q_{smoothed ground truth prob.}(w)
    and p_{prob. computed by model}(w) is minimized.
    """
    #loss_function = LabelSmoothingLoss(label_smoothing=config['label_smoothing'],
    #                                  vocabulary_size=target_dictionary.vocabulary_size)
    def __init__(self, label_smoothing, vocabulary_size, pad_index=0):
        assert 0.0 < label_smoothing <= 1.0

        super(LabelSmoothingLoss, self).__init__()

        self.pad_index = pad_index
        self.log_softmax = nn.LogSoftmax(dim=-1) # 这里走了一步log的归一化
        #`nn.LogSoftmax()` 函数在进行概率计算之前，对原始的实数值做了 `log` 计算，得到的结果再进行归一化处理
        #因为在求解损失函数的过程中，使用 `nn.LogSoftmax()` 能够使数值更加稳定，避免因数值过于小的数据导致的数值欠拟合（Underflow）的问题。
        self.criterion = nn.KLDivLoss(reduction='sum')#这是KL散度
        #哦，想起来了，KL散度就是相对熵

        smoothing_value = label_smoothing / (vocabulary_size - 2)  # exclude pad and true label
        #这里的-2应该是去掉开始符和结束符
        smoothed_targets = torch.full((vocabulary_size,), smoothing_value)
        # torch.full是一个创建张量的函数，返回一个指定形状和类型的张量，并且设置所有元素的值为指定的值。
        smoothed_targets[self.pad_index] = 0 #smoothed_targets不在实例类里面，只在class类里面
        self.register_buffer('smoothed_targets', smoothed_targets.unsqueeze(0))  # (1, vocabulary_size)

        self.confidence = 1.0 - label_smoothing

    def forward(self, outputs, targets):
        """
        outputs (FloatTensor): (batch_size, seq_len, vocabulary_size)
        targets (LongTensor): (batch_size, seq_len)
        """
        batch_size, seq_len, vocabulary_size = outputs.size()

        outputs_log_softmax = self.log_softmax(outputs)
        outputs_flat = outputs_log_softmax.view(batch_size * seq_len, vocabulary_size)
        targets_flat = targets.view(batch_size * seq_len)

        smoothed_targets = self.smoothed_targets.repeat(targets_flat.size(0), 1)#这一步开始变得不同
        #targets_flat.size(0)是64*49
        # 这一步是将 `smoothed_targets` 重复扩展至大小为 `(batch_size * seq_len, vocabulary_size)` 的张量，
        # 以便和 `outputs_flat` 的形状保持一致，方便后面计算损失值。
        # smoothed_targets: (batch_size * seq_len, vocabulary_size)

        smoothed_targets.scatter_(1, targets_flat.unsqueeze(1), self.confidence)
        # smoothed_targets: (batch_size * seq_len, vocabulary_size)
        #`scatter_` 是 PyTorch 的一个操作，可以在指定维度上，按照给定的 index，将 tensor 中的元素替换为指定的 value。
        #这句代码会把 `self.confidence` 值分配到 `smoothed_targets` 中 `targets_flat.unsqueeze(1)` 指定位置上。
        #这都是对targets进行的，目的是把one-hot的标签概率给平滑化

        smoothed_targets.masked_fill_((targets_flat == self.pad_index).unsqueeze(1), 0)
        #这里是在造掩码层，把targets中的填充的部分给等于出来，然后赋值为0，用来消除填充词的影响
        #填充词应该是0，而不是平滑后的数
        # masked_targets: (batch_size * seq_len, vocabulary_size)

        loss = self.criterion(outputs_flat, smoothed_targets)#计算KL散度
        count = (targets != self.pad_index).sum().item()#计算非填充的数据的个数，用来确保程序的正确运行

        return loss, count



# 注意到，后边还有个算accuracy准确度的

In [6]:
#计算准确率

class AccuracyMetric(nn.Module):

    def __init__(self, pad_index=0):
        super(AccuracyMetric, self).__init__()

        self.pad_index = pad_index # 这个填充我没看懂

    def forward(self, outputs, targets):

        batch_size, seq_len, vocabulary_size = outputs.size()

        outputs = outputs.view(batch_size * seq_len, vocabulary_size)
        targets = targets.view(batch_size * seq_len)

        predicts = outputs.argmax(dim=1)#按照这个维度，做一步映射，这个是没有梯度的，映射为索引值，变成one-hot编码的结果，也就是最终的输出
        corrects = predicts == targets#正确的个数

        corrects.masked_fill_((targets == self.pad_index), 0)#用这种掩码的方式，把填充词的部分改为0，使其不纳入计算

        correct_count = corrects.sum().item()
        count = (targets != self.pad_index).sum().item()#还是用掩码的方式，只计算非填充的部分

        return correct_count, count

#就这样叭，还挺好懂的，23333
#主要的东西都懂了，23333，四个文件总算总结完了，淦，人看得都傻了

# 最后的最后，考虑下训练的代码，默认的设置效果看上去不怎么好

```
python train.py --data_dir=data/example/processed --save_config=checkpoints/example_config.json --save_checkpoint=checkpoints/example_model.pth --save_log=logs/example.log --positional_encoding --layers_count=4 --heads_count=4 --epochs=300 --batch_size=32


```